In [1]:
from sklearn.ensemble import (RandomForestRegressor, IsolationForest)
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import (train_test_split, GridSearchCV, cross_val_score)
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (StandardScaler, RobustScaler, OneHotEncoder, FunctionTransformer, KBinsDiscretizer)
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.linear_model import (ElasticNet, Ridge, Lasso)
from scipy import stats
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
import AveragedModels as av
from sklearn.kernel_ridge import KernelRidge
import xgboost as xgb

In [2]:
# constants and helper methods

CONDITIONS_DICT = {"NA": 0, "NaN": 0, "nan": 0, "Po": 2, "Fa": 3, "TA": 4, "Gd":6, "Ex": 10}

# constants
CATEGORY_LABELS = {"KitchenQual":       CONDITIONS_DICT,
                    "GarageCond":       CONDITIONS_DICT,
                    "GarageQual":       CONDITIONS_DICT,
                    "ExterQual":        CONDITIONS_DICT,
                    "ExterCond":        CONDITIONS_DICT,
                    "BsmtQual":         CONDITIONS_DICT,
                    "BsmtCond":         CONDITIONS_DICT,
                    "FireplaceQu" :     CONDITIONS_DICT,
                    "HeatingQC" :       CONDITIONS_DICT,
                    "LotConfig":     {"Inside": 0, "Corner": 6, "CulDSac": 10, "FR2": 3, "FR3":4},
                    "Utilities":     {"ELO": 0, "NoSeWa": 1, "NoSewr": 2, "AllPub": 3},
                    "LandSlope":     {"Gtl": 10, "Mod": 4, "Sev": 1},
                    "LotShape":     {"Reg": 10, "IR1": 5, "IR2": 3, "IR3": 1},
                    "GarageType":     {"NA": 0, "nan": 0, "Basment": 4,  "Detchd": 1, "CarPort": 3, "BuiltIn": 5, "Attchd": 7, "2Types": 12},
                    "BldgType":     {"TwnhsI": 1, "Twnhs": 2, "TwnhsE": 3, "Duplex": 5,  "2fmCon": 7, "1Fam": 12},
                    "CentralAir":     {"N": 1, "Y": 10},
                    "Electrical":     {"Mix": 1, "FuseP": 3, "FuseF": 5,  "FuseA": 7, "SBrkr": 12},
                    "MSZoning":     {"RL": 100, "RM": 60, "C (all)": 20, "FV": 30, "RH": 30},
                    "LandContour":     {"Lvl": 100, "Low": 15, "Bnk": 25, "HLS": 5},
                    "Fence":     {"NA": 0, "MnPrv": 25, "MnWw": 15, "GdWo": 40, 'GdPrv': 100},
                    "Functional":     {"Typ": 100, "Min1": 70, "Min2": 50, "Mod": 40, "Maj1": 25, "Maj2": 20, "Min2": 10, "Sev": 5, "Sal": 1},
                    "MiscFeature":     {"NA": 0, "Shed": 30, "Gar2": 40, "Othr": 25, "TenC": 100},
                    "PavedDrive":     {"Y": 100, "P": 30, "N": 0},
                    }

CAT_COLS_TO_IGNORE = ["Functional",
                        "MiscFeature",
                        "Electrical",
                        "Fence",
                        "FireplaceQu",
                        "HeatingQC"           
                    ]

CAT_COLS = [ x for x in CATEGORY_LABELS.keys() if x not in CAT_COLS_TO_IGNORE]

# plot correlations
def plotCoorelations(df):
    # remove non_numeric features  
    corr = df.corr()
    corr.style.background_gradient(cmap='coolwarm').set_precision(2)

# define a method to use Isolation Forest for outlier detection
def outlierDetect_IsolationForest(X, outlierFraction = 0.02):
    clf = IsolationForest( behaviour = 'new', contamination = outlierFraction)
    preds = clf.fit_predict(X)
    return np.where(preds == -1)

# define a method to use Isolation Forest for outlier detection
def outlierDetect_ZScore(X, zValue = 3):
    z = np.abs(stats.zscore(X))
    return np.where(z > zValue)
    
def displayScoresExp1p(scores):
    print("Scores:", np.expm1(scores))
    print("Mean:", np.expm1(scores.mean()))
    print("standard deviation:", np.expm1(scores.std()))
    
def displayScores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("standard deviation:", scores.std())
    
def arrayStats(arrayToInspect):
    print("min:", np.amin(arrayToInspect, axis=1))
    print("max:", np.amax(arrayToInspect, axis=1))
    
def crossValidateModel(model, X, y, name="<unknown>", crossVal = 10):
    start = time.time()
    scores = cross_val_score(model, X, y, scoring = "neg_mean_absolute_error", n_jobs = -1, verbose = 4, cv = crossVal)
    end = time.time()
    elapsed_time = end - start
    print("model {0} cross_val_score took {1} seconds".format(name, elapsed_time))
    displayScores(-scores)
    
def columnsWithMissingData(X, threshold = 0.9):
    # check for null items
    null_df = X.columns[X.isnull().any()]
    null_count = X[null_df].isnull().sum()/len(X.index)
    null_count_above_threshold = null_count.loc[null_count > threshold]
    null_count_above_threshold
    
    #percentage of zero values for each numeric variable
    zero_df = X.columns[(X == 0).any()]
    zero_count = (X[zero_df] == 0).sum()/len(X.index)
    zero_count_above_threshold = zero_count.loc[zero_count > threshold]
    return pd.concat([null_count_above_threshold, zero_count_above_threshold])


In [3]:
# load housing data
iowa_file_path = '../data/train.csv'
home_data = pd.read_csv(iowa_file_path)

In [4]:
# drop outliers
home_data = home_data.drop(home_data['LotFrontage']
                                     [home_data['LotFrontage']>200].index)
home_data = home_data.drop(home_data['LotArea']
                                     [home_data['LotArea']>100000].index)
home_data = home_data.drop(home_data['BsmtFinSF1']
                                     [home_data['BsmtFinSF1']>4000].index)
home_data = home_data.drop(home_data['TotalBsmtSF']
                                     [home_data['TotalBsmtSF']>6000].index)
home_data = home_data.drop(home_data['1stFlrSF']
                                     [home_data['1stFlrSF']>4000].index)
home_data = home_data.drop(home_data.LowQualFinSF
                                     [home_data['LowQualFinSF']>550].index)

In [5]:
Y = home_data["SalePrice"]
columns_to_ignore = ["Id", "1stFlrSF", "2ndFlrSF", "GarageYrBlt", "GarageArea", "TotalBsmtSF", "TotRmsAbvGrd"]
X = home_data.drop(columns = columns_to_ignore)
X = X.drop(columns = ["SalePrice"])

In [6]:
# find missing data.. remove colums that have > 90% data missing
missingData = columnsWithMissingData(X, threshold = 0.9)
print("following columns have greater than 90% data missing or NULL \n",missingData)
print("original df shape = ", X.shape)
X = X.drop(columns = missingData.keys())
print("final df shape = ", X.shape)

following columns have greater than 90% data missing or NULL 
 Alley           0.937371
PoolQC          0.995871
MiscFeature     0.964212
LowQualFinSF    0.982794
BsmtHalfBath    0.944253
3SsnPorch       0.983482
ScreenPorch     0.920853
PoolArea        0.995871
MiscVal         0.965588
dtype: float64
original df shape =  (1453, 73)
final df shape =  (1453, 64)


In [7]:
# perform feature scaling for numerical features
# Steps: 1. scale columns requiring log tranformations & feature scaling
#        2. scale columns requiring normal numerical & feature scaling
#        3. concatenate the results of 1 & 2 and then apply PCA for dimensionality reduction
#        
numeric_features = X.select_dtypes(exclude=object) 
num_features_names = numeric_features.columns

# features that need a log transformation
log_features_names = ["LotFrontage", "LotArea", "GrLivArea", "OpenPorchSF"]

log_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value = 0) ),
    ('logscaler', FunctionTransformer(np.log1p, validate=False)),
    ('scaler', RobustScaler())])

#numeric features that require a normal transformation
numeric_features_names = [x for x in num_features_names if x not in log_features_names]

numeric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', RobustScaler())])

num_transformers=[
        ('log', log_pipeline, log_features_names),
        ('num', numeric_pipeline, numeric_features_names),  
    ]

# ensure that result is always a dense matrix
num_col_transformer = ColumnTransformer(transformers=num_transformers, sparse_threshold = 0)

pca_pipeline = Pipeline(steps = [
                       ('num_ct', num_col_transformer), # apply the columntransformation
                      # ('pca', PCA(0.99)) # apply PCA to data (log + numeric)
                       ])

print(X.shape)

X_pca_t = pca_pipeline.fit_transform(X)

(1453, 64)


In [8]:
cat_features_names = X.select_dtypes(include=object).columns
#cat_features_names = CAT_COLS

cat_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# ensure that result is always a dense matrix
X_cat_t = cat_pipeline.fit_transform(X[cat_features_names])

In [9]:
# concatenate the numerical and one hot encoded categorical data
Xt = np.concatenate((X_pca_t, X_cat_t.todense() ), axis = 1)
print("concatenated Xt shape: ", Xt.shape)

concatenated Xt shape:  (1453, 279)


In [10]:
elastic_net= ElasticNet(alpha = 0.2, l1_ratio = 0.9)
crossValidateModel(elastic_net, Xt, Y, "elasticnet")
ridge_reg = Ridge(alpha = 0.1, solver="auto")
crossValidateModel(ridge_reg, Xt, Y, "ridge")
lasso_reg = Lasso(alpha = 0.1)
crossValidateModel(lasso_reg, Xt, Y, "lasso")
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
crossValidateModel(KRR, Xt, Y, "krr")
rf = RandomForestRegressor(random_state=1, n_estimators = 300)



[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of  10 | elapsed:    2.7s remaining:   11.2s
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    2.9s remaining:    2.9s
[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed:    3.2s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    3.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.


model elasticnet cross_val_score took 3.379683494567871 seconds
Scores: [16236.80204984 15753.82340599 16529.56091252 19506.05756447
 18495.21246822 14542.5021054  15339.03756253 14781.65889051
 18917.15026522 15382.26955008]
Mean: 16548.407477478064
standard deviation: 1698.3819733469888


[Parallel(n_jobs=-1)]: Done   2 out of  10 | elapsed:    0.1s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.


model ridge cross_val_score took 0.26195216178894043 seconds
Scores: [16560.89418731 17344.85348149 18489.43993088 21188.41763533
 21404.17628041 18707.85496801 17117.2491084  16911.16276057
 21166.01485479 16126.7960025 ]
Mean: 18501.68592096879
standard deviation: 1948.7366954658362


[Parallel(n_jobs=-1)]: Done   2 out of  10 | elapsed:    2.4s remaining:   10.0s
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    2.6s remaining:    2.6s
[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed:    4.2s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    4.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.


model lasso cross_val_score took 4.47800087928772 seconds
Scores: [16689.53857017 17534.13693141 18665.79967919 21212.99524361
 21669.1024858  19150.22856014 17179.49191635 17004.85130203
 21122.16463788 16242.46366356]
Mean: 18647.077299014658
standard deviation: 1944.2122861255384


[Parallel(n_jobs=-1)]: Done   2 out of  10 | elapsed:    0.4s remaining:    1.8s
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    0.6s remaining:    0.6s


model krr cross_val_score took 0.9929986000061035 seconds
Scores: [18878.59708653 19395.77316772 20542.22202648 21675.84735331
 21199.21152988 19379.86559673 18103.81922507 16844.79217853
 21766.21012482 18715.90245376]
Mean: 19650.224074282025
standard deviation: 1537.6472048988028


[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed:    0.8s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.9s finished


In [21]:
# try xgboost
xgb_model = xgb.XGBRegressor(colsample_bytree=0.8, subsample=0.5,
                             learning_rate=0.05, max_depth=5, 
                             min_child_weight=1.8, n_estimators=700,
                             reg_alpha=0.9, reg_lambda=0.9, gamma=0.01, 
                             silent=1, random_state =7, nthread = -1, refit = True)

# elastic net seems to give the best scores. use a gridsearchCV to find the best elasticNet parameters
param_grid = [
    {'learning_rate' : [0.05, 0.2,  0.5,  0.9], 'max_depth': [ 3,  5, 7]}
]

grid_search = GridSearchCV(xgb_model, param_grid, scoring = "neg_mean_absolute_error", n_jobs = -1, verbose = 10, cv = 5)
grid_search.fit(Xt, Y)


Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   13.4s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:   19.0s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:   44.3s
[Parallel(n_jobs=-1)]: Done  20 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done  49 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done  56 out of  60 | elapsed:  3.0min remaining:   12.7s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  3.2min finished
c:\users\rajesh\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
c:\

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=0.8, gamma=0.01,
                                    importance_type='gain', learning_rate=0.05,
                                    max_delta_step=0, max_depth=5,
                                    min_child_weight=1.8, missing=None,
                                    n_estimators=700, n_jobs=1, nthread=-1,
                                    objective='reg:linear', random_state=7,
                                    refit=True, reg_alpha=0.9, reg_lambda=0.9,
                                    scale_pos_weight=1, seed=None, silent=1,
                                    subsample=0.5, verbosity=1),
             iid='warn', n_jobs=-1,
             param_grid=[{'learning_rate': [0.05, 0.2, 0.5, 0.9],
                          'max_depth'

In [22]:
print(grid_search.best_params_ )
print(grid_search.best_score_ )
print(grid_search.refit_time_ )

{'learning_rate': 0.05, 'max_depth': 3}
-14957.826186661217
2.724998950958252


In [23]:
xgb_model = grid_search.best_estimator_
print(xgb_model)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=0.01,
             importance_type='gain', learning_rate=0.05, max_delta_step=0,
             max_depth=3, min_child_weight=1.8, missing=None, n_estimators=700,
             n_jobs=1, nthread=-1, objective='reg:linear', random_state=7,
             refit=True, reg_alpha=0.9, reg_lambda=0.9, scale_pos_weight=1,
             seed=None, silent=1, subsample=0.5, verbosity=1)


In [12]:
import AveragedModels as av
avmodel = av.AveragingModels(models = (elastic_net, lasso_reg, rf, xgb_model))
avmodel.fit(Xt,Y)

(ElasticNet(alpha=0.2, copy_X=True, fit_intercept=True, l1_ratio=0.9,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=None, selection='cyclic', tol=0.0001, warm_start=False), Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False), RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=300,
                      n_jobs=None, oob_score=False, random_state=1, verbose=0,
                      warm_start=False), XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_b

c:\users\rajesh\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 246644542555.24, tolerance: 905296277.1952931
  positive)
c:\users\rajesh\appdata\local\programs\python\python37-32\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
c:\users\rajesh\appdata\local\programs\python\python37-32\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


AveragingModels(models=None)

In [24]:
# use the grid serach results to create the final predictor for the test
xgb_model.fit(Xt,Y)

#path to file you will use for predictions
test_data_path = '../data/test.csv'
test_data = pd.read_csv(test_data_path)
print(test_data.shape)
X_test = test_data.drop(columns = columns_to_ignore)
X_test = X_test.drop(columns = missingData.keys())
print(X_test.shape)
X_pca_test = pca_pipeline.transform(X_test)
# categorical colums that are OHE
X_cat_test = cat_pipeline.transform(X_test[cat_features_names])

X_final_test = np.concatenate((X_pca_test, X_cat_test.todense() ), axis = 1)
#make predictions which we will submit. 
y_pred = xgb_model.predict(X_final_test)

#The lines below shows how to save predictions in format used for competition scoring
output = pd.DataFrame({'Id': test_data.Id,
                       'SalePrice': y_pred})
output.to_csv('../data/submission.csv', index=False)

(1459, 80)
(1459, 64)
